# Overview

We're encountering an bug where the sentence chunks and their corresponding labels are not aligning. This is messing up our intrinsic evaluation results. Misalignment is ikely due to the 'nan' values of some labels in a few datasets not being parsed.

# Env Setup

In [ ]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
pip install nest_asyncio

In [ ]:
import csv
import json
import pandas as pd
import numpy as np
import re
import string
from collections import Counter
import openai
import os
from google.colab import drive
import time
import ast

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/'6.8611 Research Project'/'Colab Notebooks'

/content/drive/MyDrive/6.8611 Research Project/Colab Notebooks


In [ ]:
ls

 BC5CDR-D_devel_1.csv               ' NER with BERT.ipynb'
 BC5CDR-D_devel_2.csv                openai-test.ipynb
 Data-cleaning.ipynb                 retry_prompts.gsheet
 devel_gpt_generated_datasets/       RW-Fine-Tuning-Human-Annotated.ipynb
 Fine-Tuning-Human-Annotated.ipynb   TEST_LABEL_BUGS.ipynb
 Fine-Tuning-One-Shot.ipynb          tokens_labels.csv
 Fine-Tuning-Zero-Shot.ipynb         zero-shot-bc5cdr-chem.pynb
 intrinsic_eval_entity.ipynb        'zero_shot[FASTER].ipynb'
 intrinsic_eval.ipynb                zero-shot.pynb
 llm-annotations/


# Data Preprocessing

In [ ]:
# load the datasets into dataframes

def load_tsv_dataset(file_path):
  """
  Loads a tsv dataset. Renames thne columns to 'token' and 'label'.
  Note that renaming the columns will overwrite the first row of the dataframe
  """
  df = pd.read_csv(file_path, delimiter='\t', header=None, engine='python')
  df.columns = ['token', 'label']
  print(df.head())
  return df


In [ ]:
# load the datasets into dataframes

def load_csv_dataset(file_path):
  """
  Loads a csv dataset. Renames thne columns to 'token' and 'label'.
  Note that renaming the columns will overwrite the first row of the dataframe
  """
  df = pd.read_csv(file_path, header=None, engine='python')
  df.columns = ['token', 'label']
  print(df.head())
  return df


In [ ]:
def split_by_sentence(list_of_strings):
  sentences = []
  current_sentence = []

  for word in list_of_strings:
      current_sentence.append(word)
      if type(word) is str and word.endswith('.'):
          sentence_str = ' '.join(map(str, current_sentence))
          sentences.append(sentence_str)
          current_sentence = []

  return sentences

In [ ]:
def get_filtered_entities(df, target_label):
  """
  df (pandas dataframe): has two columns 'token' and 'label'
  target_label: 'B', 'I', or 'O' (see description above for what these signify)

  Filtering involves: removing blanks, and filtering out entities that consist
  only of punctuation, numbers, or single letters.

  Return a frequency of all filtered entities with label 'target_label'.
  """
  filtered_df = df[df['label'] == target_label]
  target_entities = filtered_df['token'].tolist() # a set of all the entities with the target label

  # regex for filtering out nonsense strings
  punctuation = re.escape(string.punctuation)
  pattern = re.compile(rf'^(?![a-zA-Z]?$)(?!\d+$)(?!^[{punctuation}]+$).+')
  target_entities = [ent for ent in target_entities if pattern.match(ent)]
  return Counter(target_entities)

# Bugs

In [ ]:
dataset = "BC5CDR-chem" #NEED TO SET

In [ ]:
devel = f'llm-annotations/datasets/{dataset}/devel.tsv'
devel_df = load_tsv_dataset(devel)

test = f'llm-annotations/datasets/{dataset}/test.tsv'
test_df = load_tsv_dataset(test)

           token label
0             22     B
1              -     I
2  oxacalcitriol     I
3     suppresses     O
4      secondary     O
         token label
0      Torsade     O
1           de     O
2      pointes     O
3  ventricular     O
4  tachycardia     O


In [ ]:
def get_chunks(all_tokens, CHUNK_SIZE = 300):
  for i in range(len(all_tokens)):
    token = all_tokens[i]
    if isinstance(token, str) and 'Torsade' in token:
      print(i, token)
    if isinstance(token,float) and str(token)=="nan":
      print(all_tokens[i-5: i+5])
      all_tokens[i]="null"

  sentences = split_by_sentence(all_tokens)

  SENTENCE_CHUNKS = []

  curr_chunk, curr_chunk_len = [], 0
  for sent in sentences:
    curr_chunk.append(sent)
    curr_chunk_len += len(sent)
    if curr_chunk_len >= CHUNK_SIZE:
      SENTENCE_CHUNKS.append(' '.join(curr_chunk))
      curr_chunk = []
      curr_chunk_len = 0

  SENTENCE_CHUNKS.append(' '.join(curr_chunk))

  return SENTENCE_CHUNKS


all_tokens = devel_df['token'].tolist()
CHUNK_SIZE = 300 # string length of the chunk
SENTENCE_CHUNKS = get_chunks(all_tokens)

print('NUM CHUNKS', len(SENTENCE_CHUNKS))

all_test_tokens = test_df['token'].tolist()
print(all_test_tokens[0:100])
CHUNK_SIZE = 300 # string length of the chunk
TEST_SENTENCE_CHUNKS = get_chunks(all_test_tokens)

print('NUM TEST CHUNKS', len(TEST_SENTENCE_CHUNKS))

14135 Torsade
99756 Torsade
99785 Torsade
NUM CHUNKS 1750
['Torsade', 'de', 'pointes', 'ventricular', 'tachycardia', 'during', 'low', 'dose', 'intermittent', 'dobutamine', 'treatment', 'in', 'a', 'patient', 'with', 'dilated', 'cardiomyopathy', 'and', 'congestive', 'heart', 'failure', '.', 'The', 'authors', 'describe', 'the', 'case', 'of', 'a', '56', '-', 'year', '-', 'old', 'woman', 'with', 'chronic', ',', 'severe', 'heart', 'failure', 'secondary', 'to', 'dilated', 'cardiomyopathy', 'and', 'absence', 'of', 'significant', 'ventricular', 'arrhythmias', 'who', 'developed', 'QT', 'prolongation', 'and', 'torsade', 'de', 'pointes', 'ventricular', 'tachycardia', 'during', 'one', 'cycle', 'of', 'intermittent', 'low', 'dose', '(', '2', '.', '5', 'mcg', '/', 'kg', 'per', 'min', ')', 'dobutamine', '.', 'This', 'report', 'of', 'torsade', 'de', 'pointes', 'ventricular', 'tachycardia', 'during', 'intermittent', 'dobutamine', 'supports', 'the', 'hypothesis', 'that', 'unpredictable', 'fatal', 'arrhyth

In [ ]:
n = len(test_df)
i = 0

TEST_SENTENCE_LABELS = []

while i < n:
  for chunk in TEST_SENTENCE_CHUNKS:
    num_tokens = len(chunk.split())
    rows = test_df.iloc[i:i+num_tokens].reset_index()
    print('rpws: ', rows)

    tokens = [str(row['token'])+ '_' + str(i) for i, row in rows.iterrows()]
    labels = [row['label'] for _, row in rows.iterrows()]

    TEST_SENTENCE_LABELS.append(dict(zip(tokens, labels)))

    print(chunk)
    print(chunk.split())
    print(len(chunk.split()), len(tokens))
    print(tokens)
    if tokens[0][0] != chunk[0]:
      print('BAD: ', i)

    i += num_tokens

Streaming output truncated to the last 5000 lines.
42  13096              ,     O
43  13097             an     O
44  13098       amoeboid     O
45  13099      protozoan     O
46  13100              .     O
47  13101             An     O
48  13102     evaluation     O
was intraperitoneally administered to the guinea pigs 30 minutes before the onset of local anoxia . Mutations that extend the specificity of the endonuclease activity of lambda terminase . The gene set in this mitochondrial genome is a subset of that of Reclinomonas americana , an amoeboid protozoan .
['was', 'intraperitoneally', 'administered', 'to', 'the', 'guinea', 'pigs', '30', 'minutes', 'before', 'the', 'onset', 'of', 'local', 'anoxia', '.', 'Mutations', 'that', 'extend', 'the', 'specificity', 'of', 'the', 'endonuclease', 'activity', 'of', 'lambda', 'terminase', '.', 'The', 'gene', 'set', 'in', 'this', 'mitochondrial', 'genome', 'is', 'a', 'subset', 'of', 'that', 'of', 'Reclinomonas', 'americana', ',', 'an', 'amoeboi

KeyboardInterrupt: ignored

## NOTE: Play around until you find a test sentence you think is strong and use that as labeled example.

In [ ]:
i = 10

print(TEST_SENTENCE_CHUNKS[i])
print()
for x,y in TEST_SENTENCE_LABELS[i].items():
  print(x.split('_')[0],y)

Quinine is known to block voltage - , calcium - and ATP - sensitive K ( + ) - channels while 4 - aminopyridine is known to block voltage - sensitive K ( + ) - channels . 2 . In the counterbalanced method , quinine attenuated morphine - induced place preference , whereas 4 - aminopyridine was ineffective .

Quinine B
is O
known O
to O
block O
voltage O
- O
, O
calcium B
- O
and O
ATP B
- O
sensitive O
K B
( O
+ O
) O
- O
channels O
while O
4 B
- I
aminopyridine I
is O
known O
to O
block O
voltage O
- O
sensitive O
K B
( O
+ O
) O
- O
channels O
. O
2 O
. O
In O
the O
counterbalanced O
method O
, O
quinine B
attenuated O
morphine B
- O
induced O
place O
preference O
, O
whereas O
4 B
- I
aminopyridine I
was O
ineffective O
. O


In [ ]:
example_text = TEST_SENTENCE_CHUNKS[i]
example_response = TEST_SENTENCE_LABELS[i]

In [ ]:
text = SENTENCE_CHUNKS[0] #text is the chunk to that is annotated by API call made in next cell

# remember to change <entity type> based on dataset in "focusing on identifying <entity type> entities" of INSTRUCTION
INSTRUCTION = f"""Given a biomedical text, perform Named Entity Recognition analysis on this text, focusing on identifying ONLY CHEMICAL ENTITIES. Individual tokens are separated by white space and must be given exactly one label: ‘B’,  ‘I’, or ‘O’, according to the BIO labeling schema. Output a JSON object with key equal to token concatenated with '_'+(index of token) and value equal to label, where (index of token) starts from 0 for the first token, 1 for the second token, and so on. Here is a labeled example to help with this task:

Example Text:
{example_text}

Example Response:
{example_response}

Now label the following text: """

prompt = f"""{INSTRUCTION}\n{text}"""
print(prompt)

Given a biomedical text, perform Named Entity Recognition analysis on this text, focusing on identifying ONLY CHEMICAL ENTITIES. Individual tokens are separated by white space and must be given exactly one label: ‘B’,  ‘I’, or ‘O’, according to the BIO labeling schema. Output a JSON object with key equal to token concatenated with '_'+(index of token) and value equal to label, where (index of token) starts from 0 for the first token, 1 for the second token, and so on. Here is a labeled example to help with this task:

Example Text:
Quinine is known to block voltage - , calcium - and ATP - sensitive K ( + ) - channels while 4 - aminopyridine is known to block voltage - sensitive K ( + ) - channels . 2 . In the counterbalanced method , quinine attenuated morphine - induced place preference , whereas 4 - aminopyridine was ineffective .

Example Response:
{'Quinine_0': 'B', 'is_1': 'O', 'known_2': 'O', 'to_3': 'O', 'block_4': 'O', 'voltage_5': 'O', '-_6': 'O', ',_7': 'O', 'calcium_8': 'B',